# Synacor Challenge

- [Preliminaries](#Preliminaries)
- [Entering the maze](#Entering-the-maze)
- [Coins puzzle](#Coins-puzzle)
- [Mystery function puzzle](#Mystery-function-puzzle)
- [Back in the maze](#Back-in-the-maze)
- [Grid puzzle](#Grid-puzzle)

The [Synacor Challenge](https://challenge.synacor.com) by [Eric Wastl](https://twitter.com/ericwastl) is a precursor to his [Advent of Code](https://adventofcode.com) series of annual puzzles.  It presents a machine specification and binary program and requires that a virtual machine be written to execute the program.  The program turns out to be an [Adventure](https://en.wikipedia.org/wiki/Colossal_Cave_Adventure)-style game.  As the game is played it is possible to gather codes (which serve as signs of progress), and certain puzzles must be solved to progress further into the game.  Not being familiar with these kinds of games, I will confess that I had to peek at others' solutions to get ideas of what to look for and do.  But, in hindsight, just enough clues are given to make it possible to get through the game, and the detective work required, combined with the cleverness of the puzzles, made for a very enjoyable time.

## Preliminaries

To start off, here's a virtual machine implemented directly from the specification.  We add a few hooks to enable some additional functionality farther down.

In [1]:
M = 32768 # modulus

(
    OP_halt,
    OP_set,
    OP_push,
    OP_pop,
    OP_eq,
    OP_gt,
    OP_jmp,
    OP_jt,
    OP_jf,
    OP_add,
    OP_mult,
    OP_mod,
    OP_and,
    OP_or,
    OP_not,
    OP_rmem,
    OP_wmem,
    OP_call,
    OP_ret,
    OP_out,
    OP_in,
    OP_noop
) = range(22)

identity = lambda char: char
devnull = lambda char: None
noop = lambda ip: None

class VM:

    def __init__(self, program):
        self.program = program[:]
        self.reset()

    def reset(self):
        self.memory = [0]*M
        self.registers = [0]*8
        self.stack = []
        self.ip = 0
        self.memory[:len(self.program)] = self.program

    def run(
        self,
        commands,
        output_hook=identity,
        echo_hook=identity,
        pre_hook=noop,
        trace_log=None
    ):
        # Runs the program until the machine halts or the input runs out.
        # Calling this method again picks up where the machine left off.
        def halt():
            raise EOFError
        def get(n):
            return n if n < M else self.registers[n-M]
        def set_register(register, value):
            self.registers[register-M] = value
        def set_memory(address, value):
            self.memory[address] = value
        def set_ip(value, push_return_address=False):
            if push_return_address:
                self.stack.append(self.ip+2)
            self.ip = value
        input = "".join(c+"\n" for c in commands)
        input_ptr = [0]
        output = [""]
        def read_char():
            if input_ptr[0] < len(input):
                c = input[input_ptr[0]]
                input_ptr[0] += 1
                e = echo_hook(c)
                if e != None:
                    output[0] += e
                return c
            else:
                raise EOFError
        def write_char(char):
            o = output_hook(char)
            if o != None:
                output[0] += o
        instructions = [
            lambda: halt(),                                                # halt
            lambda a, b: set_register(a, get(b)),                          # set
            lambda a: self.stack.append(get(a)),                           # push
            lambda a: set_register(a, self.stack.pop()),                   # pop
            lambda a, b, c: set_register(a, 1 if get(b) == get(c) else 0), # eq
            lambda a, b, c: set_register(a, 1 if get(b) > get(c) else 0),  # gt
            lambda a: set_ip(get(a)-2),                                    # jmp
            lambda a, b: set_ip(get(b)-3 if get(a) != 0 else self.ip),     # jt
            lambda a, b: set_ip(get(b)-3 if get(a) == 0 else self.ip),     # jf
            lambda a, b, c: set_register(a, (get(b)+get(c))%M),            # add
            lambda a, b, c: set_register(a, (get(b)*get(c))%M),            # mult
            lambda a, b, c: set_register(a, get(b)%get(c)),                # mod
            lambda a, b, c: set_register(a, get(b)&get(c)),                # and
            lambda a, b, c: set_register(a, get(b)|get(c)),                # or
            lambda a, b: set_register(a, (~get(b))&(M-1)),                 # not
            lambda a, b: set_register(a, self.memory[get(b)]),             # rmem
            lambda a, b: set_memory(get(a), get(b)),                       # wmem
            lambda a: set_ip(get(a)-2, push_return_address=True),          # call
            lambda: set_ip(self.stack.pop()-1),                            # ret
            lambda a: write_char(chr(get(a))),                             # out
            lambda a: set_register(a, ord(read_char())),                   # in
            lambda: None                                                   # noop
        ]
        try:
            while True:
                pre_hook(self.ip)
                function = instructions[self.memory[self.ip]]
                argcount = function.__code__.co_argcount
                if trace_log != None:
                    trace_log.append(
                        (
                            self.ip,
                            tuple(self.memory[self.ip:self.ip+1+argcount])
                        )
                    )
                function(*self.memory[self.ip+1:self.ip+1+argcount])
                self.ip += 1+argcount
        except EOFError:
            pass
        return output[0]

Load the program.

In [2]:
bytes = open("challenge.bin", "rb").read()
program = [bytes[i+1]*256+bytes[i] for i in range(0, len(bytes), 2)]
vm = VM(program)

Output formatting.  Hereinafter we will be displaying output from the virtual machine.  It will be nice to highlight input going into the machine and any codes we come across.

In [3]:
from html import escape
import re
from IPython.display import display, HTML

def highlight_codes(output):
    # Codes are strings of letters of intermixed case.
    return re.sub(
        r"\b((?:[a-z]+[A-Z]+[a-z]|[A-Z]+[a-z]+[A-Z])\w*)\b",
        "<span style='color: #F50D1B'>\\1</span>",
        output
    )

def run(commands=[], pre_hook=noop):
    output_hook = lambda char: escape(char, quote=False)
    echo_hook = (
        lambda char: f"<span style='color: white'>{output_hook(char)}</span>"
    )
    display(
        HTML(
            "<div style='background-color: #051005; padding: 1ex'>"
            + "<code style='background-color: #051005; color: #35B037'>"
            + highlight_codes(
                vm.run(
                    commands,
                    output_hook=output_hook,
                    echo_hook=echo_hook,
                    pre_hook=pre_hook
                )
            )
            + "</code>"
            + "</div>"
        )
    )

## Entering the maze

OK, let's run the machine and see what happens.

In [4]:
run()

At this point we need to start taking actions, such as...

In [5]:
complete_path = [
    "take tablet",
    "use tablet"
]

run(complete_path)

From here on out it takes a lot of interactive searching and experimenting to avoid being eaten by a [grue](https://en.wikipedia.org/wiki/Grue_(monster)).  The successful path we found has us finding an empty lantern, finding an oil can and using it to light the lantern, which then gives us enough light to proceed down dark passages, picking up a coin, and then finding ourselves facing a mysterious door.

In [6]:
path = [
    "doorway",
    "north",
    "north",
    "bridge",
    "continue",
    "down",
    "east",
    "take empty lantern",
    "west",
    "west",
    "passage",
    "ladder",
    "west",
    "south",
    "north",
    "take can",
    "use can",
    "use lantern",
    "west",
    "ladder",
    "darkness",
    "continue",
    "west",
    "west",
    "west",
    "west",
    "north",
    "take red coin",
    "north"
]

complete_path += path
run(path)

## Coins puzzle

At this point we're in a hall facing a door to the north that is locked, there's a monument with some slots, we're presented with what appears to be an equation with five unknowns, and we hold one coin.  Inserting the coin in one of the slots doesn't unlock the door, but it does fill in one of the equation unknowns, a hint that there are four more coins to be found.

In [7]:
path = [
    "east",
    "take concave coin",
    "down",
    "take corroded coin",
    "up",
    "west",
    "west",
    "take blue coin",
    "up",
    "take shiny coin",
    "down",
    "east"
]

complete_path += path
run(path)

We can put the coins in the slots, but the order matters.  Examining the coins, we can infer denominations.

In [8]:
run([
    "look red coin",
    "look concave coin",
    "look corroded coin",
    "look blue coin",
    "look shiny coin"
])

Find which permutation of the denominations satisfies the equation.

In [9]:
from itertools import permutations

next(
    filter(
        lambda p: p[0] + p[1] * p[2]**2 + p[3]**3 - p[4] == 399,
        permutations([2, 3, 5, 7, 9])
    )
)

(9, 2, 5, 7, 3)

So the order of the coins is blue, red, shiny, concave, corroded.  Inserting them in that order allows us to proceed through the door and to a teleporter.

In [10]:
path = [
    "use blue coin",
    "use red coin",
    "use shiny coin",
    "use concave coin",
    "use corroded coin",
    "north",
    "take teleporter",
    "use teleporter"
]

complete_path += path
run(path)

## Mystery function puzzle

The strange book gives us clues as to what the next puzzle is.  This puzzle was quite difficult!

In [11]:
run(["look strange book"])

It seems we need to set register 7 to some special non-zero value and try using the teleporter again.  The program does not allow registers to have non-zero values at
invocation, so we're going to have to [monkey patch](https://en.wikipedia.org/wiki/Monkey_patch) the program to set the register where its value is being tested, which means we need to be able to disassemble and examine the program.

Unfortunately it's not possible to write a function that disassembles the entire program.  Portions of the program binary are data (and it is ambiguous whether memory cells are instructions or data), and making matters worse, the program is self-modifying in places.  The disassembler below assumes we're in a well-behaved section of the program, and is good enough for our purposes.

In [12]:
instructions = [
    # name, argcount
    ("halt", 0),
    ("set",  2),
    ("push", 1),
    ("pop",  1),
    ("eq",   3),
    ("gt",   3),
    ("jmp",  1),
    ("jt",   2),
    ("jf",   2),
    ("add",  3),
    ("mult", 3),
    ("mod",  3),
    ("and",  3),
    ("or",   3),
    ("not",  2),
    ("rmem", 2),
    ("wmem", 2),
    ("call", 1),
    ("ret",  0),
    ("out",  1),
    ("in",   1),
    ("noop", 0)
]

def format_instruction(address, instruction):
    def format_arg(n):
        if instruction[0] == OP_out:
            return f"'{chr(n)}'"
        else:
            return str(n) if n < M else f"R{n-M}"
    return (
        f"{address:5}: {instructions[instruction[0]][0]:4} "
        + " ".join(format_arg(n) for n in instruction[1:])
    ).rstrip()

def disassemble(program, start, n):
    # Prints n instructions starting from address `start`.
    i = start
    j = 0
    while j < n:
        argcount = instructions[program[i]][1]
        print(format_instruction(i, program[i:i+1+argcount]))
        i += 1+argcount
        j += 1

First off, since we can't examine the entire program using a disassembler, let's trace the program while we re-run it on the path so far and see where register 7 (R7) was ever referenced.

In [13]:
vm.reset()

log = []
vm.run(complete_path, trace_log=log, output_hook=devnull, echo_hook=devnull)

for entry in log:
    if M+7 in entry[1]:
        print(format_instruction(*entry))

  521: jt   R7 1093
 5451: jf   R7 5605


We see that R7 is never set, only tested.  The test at address 521 is at program startup to ensure the register is zero.  At address 5451:

In [14]:
disassemble(program, 5451, 21)

 5451: jf   R7 5605
 5454: push R0
 5456: push R1
 5458: push R2
 5460: set  R0 28844
 5463: set  R1 1531
 5466: add  R2 9571 7680
 5470: call 1458
 5472: pop  R2
 5474: pop  R1
 5476: pop  R0
 5478: noop
 5479: noop
 5480: noop
 5481: noop
 5482: noop
 5483: set  R0 4
 5486: set  R1 1
 5489: call 6027
 5491: eq   R1 R0 6
 5495: jf   R1 5579


If R7 is zero, we jump past the above snippet and enter Synacor Headquarters and encounter the strange book as happened previously.  But if R7 is non-zero, some text is output (that's the call to subroutine 1458 at addresses 5454-5476); then subroutine 6027 is called with arguments R0=4, R1=1, and (implicitly) R7; and then the return value R0 is tested to see if it equals 6.  If so, success!  It might seem that we could simply set R0 = 6 and bypass the call to 6027 altogether, but the code that is output is not accepted by the Synacor website.  Apparently it's in some way a function of R7.  So, we must find a value of R7 that will produce R0 = 6.  Here's subroutine 6027:

In [15]:
disassemble(program, 6027, 16)

 6027: jt   R0 6035
 6030: add  R0 R1 1
 6034: ret
 6035: jt   R1 6048
 6038: add  R0 R0 32767
 6042: set  R1 R7
 6045: call 6027
 6047: ret
 6048: push R0
 6050: add  R1 R1 32767
 6054: call 6027
 6056: set  R1 R0
 6059: pop  R0
 6061: add  R0 R0 32767
 6065: call 6027
 6067: ret


Python equivalent, so that the program's call at address 5489 is equivalent to `R0 = A(4, 1, R7)`:

In [16]:
def A(m, n, k):
    if m == 0:
        return n+1
    elif n == 0:
        return A(m-1, k)
    else:
        return A(m-1, A(m, n-1))

This is a variation of the well-known and infamous [Ackermann Function](https://mathworld.wolfram.com/AckermannFunction.html).  For k = 1 it is the classic function, but here we are considering different values of k corresponding to different values of R7.  Computing the function one time using the Python definition above is out of the question due to the CPU time and stack space required, let alone multiple times with different values of k.  That we're computing modulo 32768 does not seem to make things easier.  We utilize two optimizations to create a workable implementation.  First, we derive analytic formulas for small values of m:

$$
\begin{eqnarray}
A(0, n, k) &=& n+1        \\
A(1, n, k) &=& n+k+1      \\
A(2, n, k) &=& (n+2)k+n+1
\end{eqnarray}
$$

Second, we eschew recursion and explicitly manage a stack.  The implementation below is admittedly not our idea, but we copy it here because it is so elegant compared to our original, clunkier implementation.  It uses a homogeneous stack for arguments and results, relying on the facts that results become arguments with no intervening computation and that the doubly nested call dovetails with the stack perfectly.

For efficiency in running this notebook, the k domain has been rotated so that we find the answer almost immediately.  Starting from k = 1, it takes about 10 minutes to find the answer. 

In [17]:
from itertools import chain

def A(m, n, k):
    stack = [m, n]
    while len(stack) > 1:
        n, m = stack.pop(), stack.pop()
        if m == 0:
            stack.append((n+1)%M)
        elif m == 1:
            stack.append((n+k+1)%M)
        elif m == 2:
            stack.append(((n+2)*k+n+1)%M)
        elif n == 0:
            stack.extend([m-1, k])
        else:
            stack.extend([m-1, m, n-1])
    return stack[0]

next(
    filter(
        lambda k: A(4, 1, k) == 6,
        chain(range(25730, 32768), range(1, 25730))
    )
)

25734

We've found the magic value for R7 and are finally ready to run the program from the beginning again up to the point where we use the teleporter.  Before using the teleporter we apply the patches as the strange book suggested.

In [18]:
vm.reset()

vm.run(complete_path[:-1], output_hook=devnull, echo_hook=devnull)

def patches(ip):
    if ip == 5451:
        vm.registers[7] = 25734
    elif ip == 5489:
        vm.memory[5489] = vm.memory[5490] = OP_noop
        vm.registers[0] = 6

run(["use teleporter"], pre_hook=patches)

## Back in the maze

A relatively uninteresting path to the next puzzle.

In [19]:
path = [
    "north",
    "north",
    "north",
    "north",
    "north",
    "north",
    "north",
    "east",
    "take journal",
    "west",
    "north",
    "north",
    "take orb"
]

complete_path += path
run(path)

## Grid puzzle

We've arrived at the antechamber to a grid of rooms where the value 22 has some kind of significance.  Investigation reveals the grid looks like this (north is up):

```
              ( *) --- ( 8) --- ( -) --- ( 1) --- (vault = 30)
                |        |        |        |
              ( 4) --- ( *) --- (11) --- ( *)
                |        |        |        |
              ( +) --- ( 4) --- ( -) --- (18)
                |        |        |        |
(antechamber = 22) --- ( -) --- ( 9) --- ( *)
```

It appears we need to find a path through the grid from the antechamber to the vault that forms an arithmetic expression that computes the value 30 starting from the value 22.  Two possibilities: the expression is interpreted according to the usual precedence of operations (22+4\*11 = 66), or the expression is built up incrementally (22+4\*11 = (22+4)\*11 = 286).  The latter turns out to be correct.  Reading the journal we picked up earlier confirms all this.

In [20]:
run(["look journal"])

Eyeballing the grid, we may need to visit rooms multiple times to form a correct expression, so we use breadth-first search (BFS) to examine all possible paths.  The journal hints at needing to complete the path quickly; breadth-first search will automatically find the shortest path.  A couple subtle points.  One, in keeping track of states we've previously seen, we qualify a room with the current running value since a room may be visited multiple times but with different values each time.  This level of state is sufficient: the first time we are in a particular room with a particular value, that will necessarily represent the shortest path to achieve that value at that point.  Two, we must avoid revisiting the antechamber, as the orb evaporates if we do.

In [21]:
from collections import namedtuple
from operator import add, sub, mul

grid = [
    [mul,   8, sub,   1],
    [  4, mul,  11, mul],
    [add,   4, sub,  18],
    [ 22, sub,   9, mul]
]

Node = namedtuple("Node", ["row", "col", "val"])

deltas = [
    (-1,  0, "north"),
    ( 1,  0, "south"),
    ( 0,  1, "east"),
    ( 0, -1, "west")
]

def bfs():
    start_node = Node(3, 0, 22)
    seen = {start_node}
    frontier = {start_node}
    paths = {start_node: []}
    while True:
        new_frontier = set()
        for n in frontier:  # for each node on the frontier
            for dr, dc, direction in deltas:  # for each direction to go in
                r, c = n.row+dr, n.col+dc
                if r not in range(0, 4) or c not in range(0, 4):
                    continue
                if (r, c) == (3, 0):  # avoid the antechamber
                    continue
                if type(grid[r][c]) is int:
                    m = Node(r, c, grid[n.row][n.col](n.val, grid[r][c]))
                else:
                    m = Node(r, c, n.val)
                if m not in seen:
                    paths[m] = paths[n]+[direction]
                    if m == Node(0, 3, 30):
                        return paths[m]
                    new_frontier.add(m)
        seen.update(new_frontier)
        frontier = new_frontier

path = bfs()
path

['north',
 'east',
 'east',
 'north',
 'west',
 'south',
 'east',
 'east',
 'west',
 'north',
 'north',
 'east']

As a reminder, here's the grid again:

```
              ( *) --- ( 8) --- ( -) --- ( 1) --- (vault = 30)
                |        |        |        |
              ( 4) --- ( *) --- (11) --- ( *)
                |        |        |        |
              ( +) --- ( 4) --- ( -) --- (18)
                |        |        |        |
(antechamber = 22) --- ( -) --- ( 9) --- ( *)
```

And the path we found corresponds to:

$$
(((((22+4)-11)*4)-18)-11)-1=30
$$

In [22]:
complete_path += path
run(path)

We've unlocked the vault!  We go inside and find, among other riches, a mirror.

In [23]:
path = [
    "vault",
    "take mirror",
    "use mirror"
]

complete_path += path
run(path)

We've reached the end!  As for the final code, we are looking at a mirror image of the actual code, which means we need to reverse both the characters in the code and individual characters.  The only letters and digits that have mirror counterparts are lowercase b and d, and lowercase p and q; the other characters that appear in the code are all vertically symmetrical (how fortunate).

In [24]:
def mirror_image(string):
    letters = "bpqd"
    return re.sub(
        r"[bpqd]",
        lambda m: letters[3-letters.index(m[0])],
        string[::-1]
    )

print(mirror_image("vAHwi8ivpbbA"))

Addqvi8iwHAv


And submitting that to the Synacor website confirms that that is the final code.

Thanks, Eric!